In [3]:
from pandas import concat,DataFrame
import numpy as np 
import pandas as pd

In [4]:
df = pd.read_csv("city_hour.csv") 
df=df[df['City']=='Chennai']
df.drop(columns=['City','NOx','PM10','SO2','NO','AQI_Bucket','Benzene','Toluene','Xylene'],axis=1,inplace=True)
df.head(5)

,Datetime,PM2.5,NO2,NH3,CO,O3,AQI
187751,2015-01-01 01:00:00,NaN,47.97,6.36,0.92,7.65,NaN
187752,2015-01-01 02:00:00,NaN,12.31,4.93,0.77,10.96,NaN
187753,2015-01-01 03:00:00,NaN,13.72,3.92,1.06,12.14,NaN
187754,2015-01-01 04:00:00,NaN,14.45,4.54,1.54,11.95,NaN
187755,2015-01-01 05:00:00,NaN,20.16,4.85,1.03,11.64,NaN


In [5]:
df.interpolate(limit_direction="both",inplace=True)
df.head(5)

,Datetime,PM2.5,NO2,NH3,CO,O3,AQI
187751,2015-01-01 01:00:00,428.3,47.97,6.36,0.92,7.65,333.0
187752,2015-01-01 02:00:00,428.3,12.31,4.93,0.77,10.96,333.0
187753,2015-01-01 03:00:00,428.3,13.72,3.92,1.06,12.14,333.0
187754,2015-01-01 04:00:00,428.3,14.45,4.54,1.54,11.95,333.0
187755,2015-01-01 05:00:00,428.3,20.16,4.85,1.03,11.64,333.0


In [6]:
df_test= pd.read_csv("Sensor_Data_Full.csv") 
df_test.interpolate(limit_direction="both",inplace=True)
df_test.head(5)

,Datetime,PM2.5,NO2,NH3,CO,O3,AQI
0,2021-01-17 10:00:00,45.31,11.52,133.01,1.37,3.04,90.0
1,2021-01-17 11:00:00,44.60,11.52,133.01,1.37,7.21,87.0
2,2021-01-17 12:00:00,39.12,11.52,133.01,1.37,14.03,77.0
3,2021-01-17 13:00:00,35.20,11.52,133.01,1.37,20.43,69.0
4,2021-01-17 14:00:00,32.67,11.52,133.01,1.37,21.31,65.0


In [7]:
aqi_list=df['AQI'].to_list()
aqi_list_test=df_test['AQI'].to_list()

In [8]:
def series_to_supervised(data, n_in, n_out):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('AQI(t-%d)' % (i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(1, n_out):
		cols.append(df.shift(-i))
		names += [('AQI(t+%d)' % (i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	return agg

df1= series_to_supervised(aqi_list, 4, 6)
print(df1)

df1_test= series_to_supervised(aqi_list_test, 4, 6)
print(df1_test)

       AQI(t-4)  AQI(t-3)  AQI(t-2)  AQI(t-1)  AQI(t+1)  AQI(t+2)  AQI(t+3)  \
0           NaN       NaN       NaN       NaN     333.0     333.0     333.0   
1           NaN       NaN       NaN     333.0     333.0     333.0     333.0   
2           NaN       NaN     333.0     333.0     333.0     333.0     333.0   
3           NaN     333.0     333.0     333.0     333.0     333.0     333.0   
4         333.0     333.0     333.0     333.0     333.0     333.0     333.0   
5         333.0     333.0     333.0     333.0     333.0     333.0     333.0   
6         333.0     333.0     333.0     333.0     333.0     333.0     333.0   
7         333.0     333.0     333.0     333.0     333.0     333.0     333.0   
8         333.0     333.0     333.0     333.0     333.0     333.0     333.0   
9         333.0     333.0     333.0     333.0     333.0     333.0     333.0   
10        333.0     333.0     333.0     333.0     333.0     333.0     333.0   
11        333.0     333.0     333.0     333.0     33

In [9]:
df2 = pd.concat([df.reset_index(drop=True),df1.reset_index(drop=True)], axis=1)
print(df2)

                  Datetime   PM2.5    NO2    NH3    CO      O3    AQI  \
0      2015-01-01 01:00:00  428.30  47.97   6.36  0.92    7.65  333.0   
1      2015-01-01 02:00:00  428.30  12.31   4.93  0.77   10.96  333.0   
2      2015-01-01 03:00:00  428.30  13.72   3.92  1.06   12.14  333.0   
3      2015-01-01 04:00:00  428.30  14.45   4.54  1.54   11.95  333.0   
4      2015-01-01 05:00:00  428.30  20.16   4.85  1.03   11.64  333.0   
5      2015-01-01 06:00:00  428.30   8.69   5.17  1.10   11.31  333.0   
6      2015-01-01 07:00:00  428.30  19.66   6.02  1.29   12.90  333.0   
7      2015-01-01 08:00:00  428.30  22.23   8.41  1.29   13.32  333.0   
8      2015-01-01 09:00:00  428.30  20.16   4.58  1.30   12.66  333.0   
9      2015-01-01 10:00:00  428.30   6.89   4.26  1.08   11.49  333.0   
10     2015-01-01 11:00:00  428.30  15.82   3.93  1.18   12.22  333.0   
11     2015-01-01 12:00:00  428.30  35.29   4.00  1.60   10.02  333.0   
12     2015-01-01 13:00:00  428.30   9.62   3.91  1

In [10]:
df2_test = pd.concat([df_test.reset_index(drop=True),df1_test.reset_index(drop=True)], axis=1)
print(df2_test)

                 Datetime  PM2.5     NO2     NH3     CO     O3    AQI  \
0     2021-01-17 10:00:00  45.31   11.52  133.01   1.37   3.04   90.0   
1     2021-01-17 11:00:00  44.60   11.52  133.01   1.37   7.21   87.0   
2     2021-01-17 12:00:00  39.12   11.52  133.01   1.37  14.03   77.0   
3     2021-01-17 13:00:00  35.20   11.52  133.01   1.37  20.43   69.0   
4     2021-01-17 14:00:00  32.67   11.52  133.01   1.37  21.31   65.0   
5     2021-01-17 15:00:00  28.62   11.52  133.01   1.37  19.42   56.0   
6     2021-01-17 16:00:00  22.43   11.52  133.01   1.37  24.42   44.0   
7     2021-01-17 17:00:00  18.21   11.52  133.01   1.37  13.37   36.0   
8     2021-01-17 18:00:00  16.47   11.52  133.01   1.37  17.24   32.0   
9     2021-01-17 19:00:00  17.07   11.52  133.01   1.37  17.78   34.0   
10    2021-01-17 20:00:00  24.89   11.52  133.01   1.37  17.36   47.0   
11    2021-01-17 21:00:00  25.31   11.52  133.01   1.37   5.12   49.0   
12    2021-01-17 22:00:00  20.07   11.52  133.01   

In [11]:
df2_test.head(5)

,Datetime,PM2.5,NO2,NH3,CO,O3,AQI,AQI(t-4),AQI(t-3),AQI(t-2),AQI(t-1),AQI(t+1),AQI(t+2),AQI(t+3),AQI(t+4),AQI(t+5)
0,2021-01-17 10:00:00,45.31,11.52,133.01,1.37,3.04,90.0,NaN,NaN,NaN,NaN,87.0,77.0,69.0,65.0,56.0
1,2021-01-17 11:00:00,44.60,11.52,133.01,1.37,7.21,87.0,NaN,NaN,NaN,90.0,77.0,69.0,65.0,56.0,44.0
2,2021-01-17 12:00:00,39.12,11.52,133.01,1.37,14.03,77.0,NaN,NaN,90.0,87.0,69.0,65.0,56.0,44.0,36.0
3,2021-01-17 13:00:00,35.20,11.52,133.01,1.37,20.43,69.0,NaN,90.0,87.0,77.0,65.0,56.0,44.0,36.0,32.0
4,2021-01-17 14:00:00,32.67,11.52,133.01,1.37,21.31,65.0,90.0,87.0,77.0,69.0,56.0,44.0,36.0,32.0,34.0


In [12]:
df2_test.tail(5)

,Datetime,PM2.5,NO2,NH3,CO,O3,AQI,AQI(t-4),AQI(t-3),AQI(t-2),AQI(t-1),AQI(t+1),AQI(t+2),AQI(t+3),AQI(t+4),AQI(t+5)
1689,2021-03-29 08:00:00,66.87,7.95,58.00,9.85,28.78,116.0,104.0,106.0,108.0,113.0,111.0,109.0,107.0,102.0,NaN
1690,2021-03-29 09:00:00,61.34,6.60,94.02,9.81,25.64,111.0,106.0,108.0,113.0,116.0,109.0,107.0,102.0,NaN,NaN
1691,2021-03-29 10:00:00,59.09,7.05,32.10,9.64,23.94,109.0,108.0,113.0,116.0,111.0,107.0,102.0,NaN,NaN,NaN
1692,2021-03-29 11:00:00,57.06,11.19,137.40,9.28,19.09,107.0,113.0,116.0,111.0,109.0,102.0,NaN,NaN,NaN,NaN
1693,2021-03-29 12:00:00,52.85,16.52,153.18,9.35,20.81,102.0,116.0,111.0,109.0,107.0,NaN,NaN,NaN,NaN,NaN


In [13]:
actual_values=[aqi_list_test[-5:]]
print(actual_values)

[[116.0, 111.0, 109.0, 107.0, 102.0]]


In [14]:
dropnan=True
if dropnan:
	df2.dropna(inplace=True)

In [15]:
dropnan=True
if dropnan:
	df2_test.dropna(inplace=True)

In [16]:
df2_test.head(5)

,Datetime,PM2.5,NO2,NH3,CO,O3,AQI,AQI(t-4),AQI(t-3),AQI(t-2),AQI(t-1),AQI(t+1),AQI(t+2),AQI(t+3),AQI(t+4),AQI(t+5)
4,2021-01-17 14:00:00,32.67,11.52,133.01,1.37,21.31,65.0,90.0,87.0,77.0,69.0,56.0,44.0,36.0,32.0,34.0
5,2021-01-17 15:00:00,28.62,11.52,133.01,1.37,19.42,56.0,87.0,77.0,69.0,65.0,44.0,36.0,32.0,34.0,47.0
6,2021-01-17 16:00:00,22.43,11.52,133.01,1.37,24.42,44.0,77.0,69.0,65.0,56.0,36.0,32.0,34.0,47.0,49.0
7,2021-01-17 17:00:00,18.21,11.52,133.01,1.37,13.37,36.0,69.0,65.0,56.0,44.0,32.0,34.0,47.0,49.0,39.0
8,2021-01-17 18:00:00,16.47,11.52,133.01,1.37,17.24,32.0,65.0,56.0,44.0,36.0,34.0,47.0,49.0,39.0,38.0


In [17]:
df2_test.tail(5)

,Datetime,PM2.5,NO2,NH3,CO,O3,AQI,AQI(t-4),AQI(t-3),AQI(t-2),AQI(t-1),AQI(t+1),AQI(t+2),AQI(t+3),AQI(t+4),AQI(t+5)
1684,2021-02-18 03:00:00,55.23,16.12,109.64,10.42,24.26,105.0,67.0,64.0,53.0,45.0,104.0,106.0,108.0,113.0,116.0
1685,2021-03-29 04:00:00,53.73,10.42,136.99,10.75,29.88,104.0,64.0,53.0,45.0,105.0,106.0,108.0,113.0,116.0,111.0
1686,2021-03-29 05:00:00,56.22,7.70,42.67,11.32,29.79,106.0,53.0,45.0,105.0,104.0,108.0,113.0,116.0,111.0,109.0
1687,2021-03-29 06:00:00,58.25,6.72,19.33,10.10,27.94,108.0,45.0,105.0,104.0,106.0,113.0,116.0,111.0,109.0,107.0
1688,2021-03-29 07:00:00,65.47,7.52,115.45,11.29,27.77,113.0,105.0,104.0,106.0,108.0,116.0,111.0,109.0,107.0,102.0


In [27]:
X_train=np.array(df2[['PM2.5','NO2','NH3','CO','O3','AQI','AQI(t-4)','AQI(t-3)','AQI(t-2)','AQI(t-1)']])
y_train=np.array(df2[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

X_test=np.array(df2_test[['PM2.5','NO2','NH3','CO','O3','AQI','AQI(t-4)','AQI(t-3)','AQI(t-2)','AQI(t-1)']])
y_test=np.array(df2_test[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

In [28]:
from sklearn import linear_model
lin_reg = linear_model.LinearRegression() 
lin_reg.fit(X_train, y_train) 

LinearRegression()

In [29]:
rows,cols=X_test.shape
y_pred=lin_reg.predict(X_test)
pred=y_pred[rows-1]
print('Forecasted AQI:')
print(pred)

Forecasted AQI:
[116.80783462 120.58230426 124.18357704 127.90888279 130.59031658]


In [30]:
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(actual_values,[pred]))
print('RMSE:')
print(rmse)

RMSE:
17.762892474541093


In [48]:
#PM2.5 removed
X_train=np.array(df2[['NO2','NH3','CO','O3','AQI','AQI(t-4)','AQI(t-3)','AQI(t-2)','AQI(t-1)']])
y_train=np.array(df2[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

X_test=np.array(df2_test[['NO2','NH3','CO','O3','AQI','AQI(t-4)','AQI(t-3)','AQI(t-2)','AQI(t-1)']])
y_test=np.array(df2_test[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

from sklearn import linear_model
lin_reg = linear_model.LinearRegression() 
lin_reg.fit(X_train, y_train) 

rows,cols=X_test.shape
y_pred=lin_reg.predict(X_test)
pred=y_pred[rows-1]
print('Forecasted AQI:')
print(pred)

from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(actual_values,[pred]))
print('RMSE:')
print(rmse)

Forecasted AQI:
[115.26567359 117.64472085 119.87977952 122.30990823 123.88942873]
RMSE:
18.419319821221784


In [32]:
#NO2 removed
X_train=np.array(df2[['PM2.5','NH3','CO','O3','AQI','AQI(t-4)','AQI(t-3)','AQI(t-2)','AQI(t-1)']])
y_train=np.array(df2[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

X_test=np.array(df2_test[['PM2.5','NH3','CO','O3','AQI','AQI(t-4)','AQI(t-3)','AQI(t-2)','AQI(t-1)']])
y_test=np.array(df2_test[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

from sklearn import linear_model
lin_reg = linear_model.LinearRegression() 
lin_reg.fit(X_train, y_train) 

rows,cols=X_test.shape
y_pred=lin_reg.predict(X_test)
pred=y_pred[rows-1]
print('Forecasted AQI:')
print(pred)

from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(actual_values,[pred]))
print('RMSE:')
print(rmse)

Forecasted AQI:
[117.06889322 121.07839925 124.91217049 128.86146735 131.71828908]
RMSE:
18.53113215364165


In [52]:
#NH3 removed
X_train=np.array(df2[['PM2.5','NO2','CO','O3','AQI','AQI(t-4)','AQI(t-3)','AQI(t-2)','AQI(t-1)']])
y_train=np.array(df2[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

X_test=np.array(df2_test[['PM2.5','NO2','CO','O3','AQI','AQI(t-4)','AQI(t-3)','AQI(t-2)','AQI(t-1)']])
y_test=np.array(df2_test[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

from sklearn import linear_model
lin_reg = linear_model.LinearRegression() 
lin_reg.fit(X_train, y_train) 

rows,cols=X_test.shape
y_pred=lin_reg.predict(X_test)
pred=y_pred[rows-1]
print('Forecasted AQI:')
print(pred)

from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(actual_values,[pred]))
print('RMSE:')
print(rmse)

Forecasted AQI:
[116.73144799 120.42636844 123.9636274  127.63959505 130.26953889]
RMSE:
17.84791887952771


In [62]:
#CO removed
X_train=np.array(df2[['PM2.5','NO2','NH3','O3','AQI','AQI(t-4)','AQI(t-3)','AQI(t-2)','AQI(t-1)']])
y_train=np.array(df2[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

X_test=np.array(df2_test[['PM2.5','NO2','NH3','O3','AQI','AQI(t-4)','AQI(t-3)','AQI(t-2)','AQI(t-1)']])
y_test=np.array(df2_test[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

from sklearn import linear_model
lin_reg = linear_model.LinearRegression() 
lin_reg.fit(X_train, y_train) 

rows,cols=X_test.shape
y_pred=lin_reg.predict(X_test)
pred=y_pred[rows-1]
print('Forecasted AQI:')
print(pred)

from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(actual_values,[pred]))
print('RMSE:')
print(rmse)

Forecasted AQI:
[113.58330258 114.00087275 114.30684756 114.58246279 115.13842072]
RMSE:
17.980315030966143


In [35]:
#O3 removed
X_train=np.array(df2[['PM2.5','NO2','NH3','CO','AQI','AQI(t-4)','AQI(t-3)','AQI(t-2)','AQI(t-1)']])
y_train=np.array(df2[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

X_test=np.array(df2_test[['PM2.5','NO2','NH3','CO','AQI','AQI(t-4)','AQI(t-3)','AQI(t-2)','AQI(t-1)']])
y_test=np.array(df2_test[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

from sklearn import linear_model
lin_reg = linear_model.LinearRegression() 
lin_reg.fit(X_train, y_train) 

rows,cols=X_test.shape
y_pred=lin_reg.predict(X_test)
pred=y_pred[rows-1]
print('Forecasted AQI:')
print(pred)

from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(actual_values,[pred]))
print('RMSE:')
print(rmse)

Forecasted AQI:
[116.747452   120.48551752 124.06106438 127.76956906 130.43583038]
RMSE:
17.648460757412224


In [36]:
#AQI(t-4) removed
X_train=np.array(df2[['PM2.5','NO2','NH3','CO','O3','AQI','AQI(t-3)','AQI(t-2)','AQI(t-1)']])
y_train=np.array(df2[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

X_test=np.array(df2_test[['PM2.5','NO2','NH3','CO','O3','AQI','AQI(t-3)','AQI(t-2)','AQI(t-1)']])
y_test=np.array(df2_test[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

from sklearn import linear_model
lin_reg = linear_model.LinearRegression() 
lin_reg.fit(X_train, y_train) 

rows,cols=X_test.shape
y_pred=lin_reg.predict(X_test)
pred=y_pred[rows-1]
print('Forecasted AQI:')
print(pred)

from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(actual_values,[pred]))
print('RMSE:')
print(rmse)

Forecasted AQI:
[116.80754878 120.58093343 124.18064346 127.90470701 130.59913596]
RMSE:
17.7640969886654


In [65]:
#AQI(t-3) removed
X_train=np.array(df2[['PM2.5','NO2','NH3','CO','O3','AQI','AQI(t-2)','AQI(t-1)']])
y_train=np.array(df2[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

X_test=np.array(df2_test[['PM2.5','NO2','NH3','CO','O3','AQI','AQI(t-2)','AQI(t-1)']])
y_test=np.array(df2_test[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

from sklearn import linear_model
lin_reg = linear_model.LinearRegression() 
lin_reg.fit(X_train, y_train) 

rows,cols=X_test.shape
y_pred=lin_reg.predict(X_test)
pred=y_pred[rows-1]
print('Forecasted AQI:')
print(pred)


from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(actual_values,[pred]))
print('RMSE:')
print(rmse)

Forecasted AQI:
[116.76505427 120.52861467 124.08297751 127.74171466 130.38489201]
RMSE:
17.87126910001612


In [64]:
#AQI(t-2) removed
X_train=np.array(df2[['PM2.5','NO2','NH3','CO','O3','AQI','AQI(t-1)']])
y_train=np.array(df2[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

X_test=np.array(df2_test[['PM2.5','NO2','NH3','CO','O3','AQI','AQI(t-1)']])
y_test=np.array(df2_test[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

from sklearn import linear_model
lin_reg = linear_model.LinearRegression() 
lin_reg.fit(X_train, y_train) 

rows,cols=X_test.shape
y_pred=lin_reg.predict(X_test)
pred=y_pred[rows-1]
print('Forecasted AQI:')
print(pred)

from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(actual_values,[pred]))
print('RMSE:')
print(rmse)

Forecasted AQI:
[116.72179027 120.46842227 124.02205787 127.66424535 130.28139198]
RMSE:
17.802000345165947


In [39]:
#AQI(t-1) removed
X_train=np.array(df2[['PM2.5','NO2','NH3','CO','O3','AQI']])
y_train=np.array(df2[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

X_test=np.array(df2_test[['PM2.5','NO2','NH3','CO','O3','AQI']])
y_test=np.array(df2_test[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

from sklearn import linear_model
lin_reg = linear_model.LinearRegression() 
lin_reg.fit(X_train, y_train) 

rows,cols=X_test.shape
y_pred=lin_reg.predict(X_test)
pred=y_pred[rows-1]
print('Forecasted AQI:')
print(pred)

from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(actual_values,[pred]))
print('RMSE:')
print(rmse)

Forecasted AQI:
[117.44972514 121.48799558 125.00215355 128.38528153 130.98408614]
RMSE:
18.251444811706232


In [40]:
#AQI removed
X_train=np.array(df2[['PM2.5','NO2','NH3','CO','O3']])
y_train=np.array(df2[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

X_test=np.array(df2_test[['PM2.5','NO2','NH3','CO','O3']])
y_test=np.array(df2_test[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

from sklearn import linear_model
lin_reg = linear_model.LinearRegression() 
lin_reg.fit(X_train, y_train) 

rows,cols=X_test.shape
y_pred=lin_reg.predict(X_test)
pred=y_pred[rows-1]
print('Forecasted AQI:')
print(pred)

from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(actual_values,[pred]))
print('RMSE:')
print(rmse)

Forecasted AQI:
[190.38886872 190.75192403 190.62930337 190.49972731 189.7384922 ]
RMSE:
81.52013716554966
